In [1]:
import os
from aicsimageio.writers import OmeZarrWriter
from aicsimageio import AICSImage
from aicsimageio.dimensions import DimensionNames, DEFAULT_CHUNK_DIMS

In [2]:
chunk_dims= [
    DimensionNames.SpatialZ,
    DimensionNames.SpatialY,
    DimensionNames.SpatialX,
    DimensionNames.Samples,
]
filepath = "//allen/aics/assay-dev/MicroscopyData/Frick/2022/20220217/LLS/AD00000198/Lamin_multi-06-Deskew-28.czi"
# filepath = "E:\\data\\AICS-11_16415.ome.tif"
img = AICSImage(filepath, chunk_dims=chunk_dims)

In [3]:
from distributed import LocalCluster, Client
cluster = LocalCluster(processes=True)  # threaded instead of multiprocess
client = Client(cluster)

In [4]:
# print some data about the image we loaded
scenes = img.scenes
print(scenes)
print(str(len(scenes)))
data = img.get_image_dask_data("TCZYX")
print(data.shape)
print(img.channel_names)
print(img.physical_pixel_sizes)
print(data)

('Image:0',)
1
(241, 2, 150, 1297, 2048)
['LatticeLightsheet', 'LatticeLightsheet_2']
PhysicalPixelSizes(Z=0.4, Y=0.14499219272808386, X=0.14499219272808386)
dask.array<transpose, shape=(241, 2, 150, 1297, 2048), dtype=uint16, chunksize=(1, 1, 150, 1297, 2048), chunktype=numpy.ndarray>


In [ ]:
# TODO: add generated thumbnail image with name "thumbnail" as 2d rgb data? or 3 channels?
# TODO compression?
# TODO loop over scenes!

# construct some per-channel lists to feed in to the writer.
# hardcoding to 9 for now
channel_colors = [0xff0000, 0x00ff00, 0x0000ff, 0xffff00, 0xff00ff, 0x00ffff, 0x880000, 0x008800, 0x000088]
pps = img.physical_pixel_sizes
cn = img.channel_names

mypath = "\\\\allen\\aics\\animated-cell\\Dan\\Lamin2.zarr"
# TODO: for all scenes in file:
scenename = scenes[0]
scenename = scenename.replace(":","_")

print(scenename)
chunksize = (1,1,1)+data.shape[-2:]
print(chunksize)

writer = OmeZarrWriter(mypath)

writer.write_image(
    image_data=data,  # : types.ArrayLike,  # must be 5D TCZYX
    image_name=scenename,  #: str,
    physical_pixel_sizes=pps, # : Optional[types.PhysicalPixelSizes],
    channel_names=cn, # : Optional[List[str]],
    channel_colors=channel_colors, # : Optional[List[int]],
    scale_num_levels=2, # : int = 1,
    scale_factor=4.0, #  : float = 2.0,
)
print(os.listdir(mypath))

Image_0
(1, 1, 1, 1297, 2048)


C:\Users\danielt\Anaconda3\envs\aicsimageio\lib\site-packages\zarr\creation.py:230: UserWarning: ignoring keyword argument 'array_key'
  warn('ignoring keyword argument %r' % k)
C:\Users\danielt\Anaconda3\envs\aicsimageio\lib\site-packages\zarr\creation.py:230: UserWarning: ignoring keyword argument 'array_key'
  warn('ignoring keyword argument %r' % k)


In [5]:
from ome_zarr.reader import Multiscales, Reader
from ome_zarr.io import parse_url
mypath = "\\\\allen\\aics\\animated-cell\\Dan\\Lamin_multi-06-Deskew-28.zarr"
# TODO: for all scenes in file:
scenename = scenes[0]
scenename = scenename.replace(":","_")

reader = Reader(parse_url(mypath + "/" + scenename))
node = list(reader())[0]
# levels
print(len(node.data))
for i in range(len(node.data)):
    print("shape ", node.data[i].shape)
    print("chunks ", node.data[i].chunks)

# assert node.data[0].chunks == ((1,), (2,), (1,), (128, 128), (128, 128))

2
shape  (241, 2, 150, 1297, 2048)
chunks  ((1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1), (1, 1), (4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 2), (1297,), (2048,))
shape  (241, 2, 150, 325, 512)
chunks  ((1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [6]:
import nbvv
level = 1
levelxyscale = 4
nbvv.volshow(node.data[level][1][0:2].compute(), spacing=(img.physical_pixel_sizes.X*levelxyscale,
        img.physical_pixel_sizes.Y*levelxyscale,
        img.physical_pixel_sizes.Z))

VolumeWidget(dimensions={'tile_width': 204, 'tile_height': 128, 'rows': 16, 'cols': 10, 'atlas_width': 2040, '…